In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models

# 1. Load the trained model
model = models.resnet101(pretrained=False)
model.load_state_dict(torch.load("best_model.pth")) # Replace with your saved model path
model.eval()  # Set the model to evaluation mode

# 2. Prepare the test dataset and DataLoader
from torchvision import datasets, transforms

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Ensure the input size matches the model
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset = datasets.ImageFolder(root="C:\\Users\\Acer\\Desktop\\Model\\test", transform=test_transforms)  # Update with your test data path
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 3. Perform testing and collect predictions
all_preds = []  # Predicted class indices
all_labels = []  # True class indices
all_probs = []  # Probabilities for all classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

with torch.no_grad():  # Disable gradient computation for efficiency
    for images, labels in test_loader:
        images = images.to(device)  # Move images to the device
        labels = labels.to(device)  # Move labels to the device

        outputs = model(images)  # Forward pass
        probs = torch.softmax(outputs, dim=1)  # Compute probabilities for all classes
        preds = torch.argmax(probs, dim=1)  # Get predicted class indices

        all_probs.extend(probs.cpu().numpy())  # Move probabilities to CPU and convert to numpy
        all_preds.extend(preds.cpu().numpy())  # Move predictions to CPU and convert to numpy
        all_labels.extend(labels.cpu().numpy())  # Move true labels to CPU and convert to numpy

# 4. Compute metrics
# Convert to numpy arrays for sklearn metrics
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_probs = np.array(all_probs)

# Multi-class Precision, Recall, and F1-Score (macro-averaged)
precision = precision_score(all_labels, all_preds, average="macro")
recall = recall_score(all_labels, all_preds, average="macro")
f1 = f1_score(all_labels, all_preds, average="macro")

print(f"Precision (macro): {precision:.2f}")
print(f"Recall (macro): {recall:.2f}")
print(f"F1 Score (macro): {f1:.2f}")

# 5. Plot ROC Curve for each class
n_classes = all_probs.shape[1]  # Number of classes
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve((all_labels == i).astype(int), all_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curves
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f"Class {i} (AUC = {roc_auc[i]:.2f})")
plt.plot([0, 1], [0, 1], "k--", label="Random Guess")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve for Multi-Class Classification")
plt.legend(loc="best")
plt.grid()
plt.show()